## IMPORT modules

In [3]:
import tensorflow as tf


In [4]:
import os
import shutil
from sklearn.model_selection import train_test_split

## data preparation

In [5]:
data_dir = '/content/drive/MyDrive/Face_Mask_Detection_Dataset_MaskNet'
train_dir= '/content/drive/MyDrive/Face_Mask_Detection_Dataset_MaskNet/train_1'
test_dir = '/content/drive/MyDrive/Face_Mask_Detection_Dataset_MaskNet/test_1'

In [6]:
classes = ['with_mask','without_mask']

In [7]:
for cls in classes:
  os.makedirs(os.path.join(train_dir,cls))
  os.makedirs(os.path.join(test_dir,cls))

In [8]:
for cls in classes:
  class_path = os.path.join(data_dir,cls)
  images = os.listdir(class_path)
  # split the images
  train_images, test_images = train_test_split(images, test_size=0.2, random_state=42)



  # move images to train test directories
  for img in  train_images:
    shutil.copy(os.path.join(class_path,img),os.path.join(train_dir,cls,img))
  for img in test_images:
    shutil.copy(os.path.join(class_path,img),os.path.join(test_dir,cls,img))

FileNotFoundError: [WinError 3] The system cannot find the path specified: '/content/drive/MyDrive/Face_Mask_Detection_Dataset_MaskNet\\with_mask'

 # Load the Data Using ImageDataGenerator
Now that the dataset is split, load it using ImageDataGenerator:

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_aug = ImageDataGenerator( rescale=1.0/255,  # Normalize pixel values
    rotation_range=20, width_shift_range=0.2, height_shift_range=0.2,
    shear_range=0.2, zoom_range=0.2, horizontal_flip=True)



test_aug = ImageDataGenerator(rescale=1.0/255) #only scaling for test data


In [ ]:
# generators

train_gen = train_aug.flow_from_directory(train_dir,target_size=(128,128),batch_size=32,class_mode='binary')

test_gen = test_aug.flow_from_directory(test_dir,target_size = (128,128),batch_size=32,class_mode='binary')

# build a basic CNN model

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Dropout,Input
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
model_1 = Sequential([
    Input(shape=(128,128,3)),
    Conv2D(150,3,activation='relu'),
    MaxPooling2D((2,2)),
    Conv2D(100,3,activation='relu'),
    MaxPooling2D((2,2)),
    Conv2D(50,3,activation='relu'),
    MaxPooling2D((2,2)),
    Flatten(),
    Dense(50,activation='relu'),
    Dropout(0.5),
    Dense(1,activation='sigmoid')
])

model_1.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
early_Stopping= EarlyStopping(monitor='val_loss',patience=3,verbose=1,restore_best_weights=True)

## model training

In [ ]:
hist = model_1.fit(train_gen,epochs=10,validation_data=test_gen,verbose=1)

### lets build another model with some paramter tunings

In [ ]:
model_2 = Sequential([
    Input(shape=(128,128,3)),
    Conv2D(150,3,activation='relu'),
    MaxPooling2D((2,2)),
    Conv2D(100,3,activation='relu'),
    MaxPooling2D((2,2)),
    Conv2D(50,3,activation='relu'),
    MaxPooling2D((2,2)),
    Flatten(),
    Dense(50,activation='relu'),
    Dropout(0.5),
    Dense(1,activation='sigmoid')
])

model_2.compile(tf.keras.optimizers.Adam(learning_rate=0.0001),loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
hist_2 = model_2.fit(train_gen,epochs=10,validation_data=test_gen,verbose=1,callbacks=[early_Stopping])

In [ ]:
hist_2.history

In [ ]:
model_2.evaluate(test_gen)

In [ ]:
pred_cnn = model_2.predict(test_gen)

In [ ]:
pred_cnn

In [ ]:
pred_model_2 = []

for x in pred_cnn:
  if x>0.5:
    pred_model_2.append(1)
  else:
    pred_model_2.append(0)

In [ ]:
pred_model_2

In [ ]:
test_gen.class_mode

In [ ]:
test_gen.class_indices

## lets also use a transfer model

In [ ]:
from tensorflow.keras.applications import VGG16

In [ ]:
base_model= VGG16(weights='imagenet',include_top=False,input_shape=(128,128,3))

In [ ]:
for layer in base_model.layers:
    layer.trainable = False


## model building using vgg 

In [ ]:
mode_vgg = Sequential([
    base_model,
    Flatten(),
    Dense(200,activation='relu'),
    Dense(50,activation='relu'),
    Dropout(0.5),
    Dense(1,activation='sigmoid')
])


mode_vgg.compile(tf.keras.optimizers.Adam(learning_rate=0.0001),loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
mode_vgg.summary()

In [ ]:
hist_vgg = mode_vgg.fit(train_gen,epochs=10,validation_data=test_gen,verbose=1,callbacks=[early_Stopping])

In [ ]:
hist_vgg.history

In [ ]:
import matplotlib.pyplot as plt

# Plot accuracy
plt.plot(hist_vgg.history['accuracy'], label='Train Accuracy')
plt.plot(hist_vgg.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.title('Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.show()

# Plot loss
plt.plot(hist_vgg.history['loss'], label='Train Loss')
plt.plot(hist_vgg.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title('Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.show()


# predictions


In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

img_path = "/content/drive/MyDrive/Face_Mask_Detection_Dataset_MaskNet/test_1/with_mask/B005.jpg"

load_img = image.load_img(img_path,target_size=(128,128))

img_array = image.img_to_array(load_img)/255

img_array = np.expand_dims(img_array,0)

predictions = mode_vgg.predict(img_array)

predictions

In [ ]:
predictions[0][0]

In [ ]:
if predictions[0][0]<0.5:
  print('with mask')
else:
  print('without mask')

# save the model

In [ ]:
mode_vgg.save('face_mask_detection_model.h5')

# deploy

In [ ]:
!pip install gradio

In [ ]:
import gradio as gr
from tensorflow.keras.models import load_model
import numpy as np
from tensorflow.keras.preprocessing import image

In [ ]:
# model_pred = load_model('face_mask_detection_model.h5')
# labels =  {0: 'With Mask', 1: 'Without Mask'}
# def predict(img):
#   load_img = image.load_img(img,target_size=(128,128))

#   img_array = image.img_to_array(load_img)/255

#   img_array = np.expand_dims(img_array,0)


#   prediction = model_pred.predict(img_array)

#   if prediction[0][0]<0.5:
#     return labels[0]
#   else:
#     return labels[1]


from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the model globally
model_pred = load_model('face_mask_detection_model.h5')
labels = {0: 'With Mask', 1: 'Without Mask'}



def predict_img(img):
    try:
        # If `img` is a PIL Image, use it directly; no need to load from path
        img = img.resize((128, 128))  # Resize to model input size
        img_array = image.img_to_array(img) / 255.0  # Normalize the image
        img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

        # Make prediction
        prediction = model_pred.predict(img_array)

        # Map prediction to label
        if prediction[0][0] < 0.5:
            return labels[0]
        else:
            return labels[1]
    except Exception as e:
        return f"Error: {str(e)}"


In [ ]:
interface = gr.Interface(
    fn=predict_img,  # Function to call
    inputs=gr.Image(type="pil"),  # Input type (PIL image)
    outputs="text",  # Output type (text)
    title="Face Mask Detection",
    description="Upload an image to detect if the person is wearing a mask or not."
)

In [ ]:
interface.launch()